In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Stops auto-scrolling so entire output is visible: see https://stackoverflow.com/a/41646403

In [ ]:
# Default parameter values. They will be overwritten by papermill notebook parameters.
# This cell must carry the tag "parameters" in its metadata.
from pathlib import Path
import pickle
import codecs

innereye_path = Path.cwd().parent.parent.parent
train_metrics_csv = ""
val_metrics_csv = innereye_path / 'Tests' / 'ML' / 'reports' / 'val_metrics_classification.csv'
test_metrics_csv = innereye_path / 'Tests' / 'ML' / 'reports' / 'test_metrics_classification.csv'
number_best_and_worst_performing = 20
config = ""
is_crossval_report = False

In [ ]:
import sys
print(f"Adding to path: {innereye_path}")
if str(innereye_path) not in sys.path:
    sys.path.append(str(innereye_path))

%matplotlib inline
import matplotlib.pyplot as plt

config = pickle.loads(codecs.decode(config.encode(), "base64"))

from InnerEye.ML.common import ModelExecutionMode
from InnerEye.ML.reports.notebook_report import print_header
from InnerEye.ML.reports.classification_report import plot_pr_and_roc_curves_from_csv, \
    print_k_best_and_worst_performing, print_metrics_for_all_prediction_targets, \
    plot_k_best_and_worst_performing

import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (20, 10)

#convert params to Path
train_metrics_csv = Path(train_metrics_csv)
val_metrics_csv = Path(val_metrics_csv)
test_metrics_csv = Path(test_metrics_csv)

# Metrics
## Train Set

In [ ]:
if train_metrics_csv.is_file():
    print_metrics_for_all_prediction_targets(csv_to_set_optimal_threshold=train_metrics_csv,
                                             data_split_to_set_optimal_threshold=ModelExecutionMode.TRAIN,
                                             csv_to_compute_metrics=train_metrics_csv,
                                             data_split_to_compute_metrics=ModelExecutionMode.TRAIN,
                                             config=config, is_thresholded=False, is_crossval_report=is_crossval_report)

## Validation Set

In [ ]:
if val_metrics_csv.is_file():
    print_metrics_for_all_prediction_targets(csv_to_set_optimal_threshold=val_metrics_csv,
                                             data_split_to_set_optimal_threshold=ModelExecutionMode.VAL,
                                             csv_to_compute_metrics=val_metrics_csv,
                                             data_split_to_compute_metrics=ModelExecutionMode.VAL,
                                             config=config, is_thresholded=False, is_crossval_report=is_crossval_report)

## Test Set

In [ ]:
if val_metrics_csv.is_file() and test_metrics_csv.is_file():
    print_metrics_for_all_prediction_targets(csv_to_set_optimal_threshold=val_metrics_csv,
                                             data_split_to_set_optimal_threshold=ModelExecutionMode.VAL,
                                             csv_to_compute_metrics=test_metrics_csv,
                                             data_split_to_compute_metrics=ModelExecutionMode.TEST,
                                             config=config, is_thresholded=False, is_crossval_report=is_crossval_report)

# ROC and PR curves
## Train Set

In [ ]:
if train_metrics_csv.is_file():
    plot_pr_and_roc_curves_from_csv(metrics_csv=train_metrics_csv, data_split=ModelExecutionMode.TRAIN, config=config,
                                    is_crossval_report=is_crossval_report)

## Validation set

In [ ]:
if val_metrics_csv.is_file():
    plot_pr_and_roc_curves_from_csv(metrics_csv=val_metrics_csv, data_split=ModelExecutionMode.VAL, config=config,
                                    is_crossval_report=is_crossval_report)

## Test set

In [ ]:
if test_metrics_csv.is_file():
    plot_pr_and_roc_curves_from_csv(metrics_csv=test_metrics_csv, data_split=ModelExecutionMode.TEST, config=config,
                                    is_crossval_report=is_crossval_report)

# Best and worst samples by ID

In [ ]:
if not is_crossval_report and val_metrics_csv.is_file() and test_metrics_csv.is_file():
    for prediction_target in config.class_names:
        print_header(f"Class: {prediction_target}", level=3)
        print_k_best_and_worst_performing(val_metrics_csv=val_metrics_csv, test_metrics_csv=test_metrics_csv,
                                          k=number_best_and_worst_performing,
                                          prediction_target=prediction_target)

# Plot best and worst sample images

In [ ]:
if not is_crossval_report and val_metrics_csv.is_file() and test_metrics_csv.is_file():
    for prediction_target in config.class_names:
        print_header(f"Class: {prediction_target}", level=3)
        plot_k_best_and_worst_performing(val_metrics_csv=val_metrics_csv, test_metrics_csv=test_metrics_csv,
                                         k=number_best_and_worst_performing, prediction_target=prediction_target,
                                         config=config)